# Reproduce Results from Perdikaris with QUEENS

The purpose of this notebook is the reproduction of some of the results presented in [Perdikaris et al. 2017](http://rspa.royalsocietypublishing.org/content/473/2198/20160751). The code published along with the original publication can be found [here](https://github.com/paraklas/NARGP).

## 1D Sinus function

The first example is the following multi-fidelity expression:

The low-fidelity version is defined as:

$f_l(x) = sin(8\pi x)$

And the high fidelity version as:

$f_h(x) = (x- \sqrt(2)f_l^2(x))$

Below, we will build multi-fidelity surrogate models using the nonlinear autoregressive scheme put forth in [[1](http://rspa.royalsocietypublishing.org/content/473/2198/20160751)] and for comparison the original autoregressive scheme proposed by Kennedy and O'Hagan in [[2](https://academic.oup.com/biomet/article/87/1/1/221217)]. The respective surrogate models will be referred to as NARGP and ICM.


### Input File

In [9]:
# Define multi-fidelity model
options = { 
    "output_dir" : "/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output",
    "experiment-name" : "perdikaris_icm_test",
    "database"  :{
      "address"   : "localhost:27017"
    },
    "method": {
      "method_name": "monte_carlo",
      "method_options" :{
        "seed" : 44,
        "num_samples" : 500,
        "model" : "mf_gp_icm_surrogate_model"
      }
    },
    "mf_gp_icm_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression"
    },
    "mf_gp_regression" : {
      "type" : "mf_icm_gp_approximation_gpy",
    },
    "mf_gp_nargp_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface2",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
     "gp_surrogate_model" : {
      "type" : "datafit_surrogate_model",
      "interface" : "approximation_interface3",
      "parameters" : "parameters",
      "subordinate_model" : "hifi_sin",
      "subordinate_iterator" : "hifi_model_iterator"
    },
    "approximation_interface2" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression2"
    },
    "approximation_interface3" : {
      "type" : "approximation_interface",
      "approximation" : "gp_regression"
    },
    "mf_gp_regression2" : {
      "type" : "mf_nar_gp_approximation_gpy_2_levels",
    },
     "gp_regression" : {
      "type" : "gp_approximation_gpy",
    },
    "thruth_model_iterator" : {
      "method_name": "lhs_mf",
      "method_options" :{
        "seed" : 42,
        "num_samples" : [50, 10],
        "num_iterations" :10,
        "mode" : "nested"
      },
    },
    "hifi_model_iterator" : {
      "method_name": "lhs",
      "method_options" :{
        "seed" : 42,
        "num_samples" : 23,
        "num_iterations" :10,
      },
    },
    "truth_model" :{
      "type" : "multi_fidelity_model",
      "model_hierarchy" : ["lofi_sin", "hifi_sin"],
      "eval_cost_per_level" :[1, 1],
      "parameters" : "parameters"
    },
    "hifi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_hifi",
      "parameters" : "parameters"
    },
    "lofi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_lofi"
    },
    "interface_lofi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_lofi.py"
    },
    "interface_hifi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_hifi.py"
    },
    "parameters" : {
        "x" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1,
            "distribution" : "uniform",
            "distribution_parameter" : [0,1]
        }
    }
}



### Run QUEENS


In [2]:
#from pqueens.iterators.iterator import Iterator
# build iterator
#my_iterator = Iterator.from_config_create_iterator(options)
# perform analysis
#my_iterator.run()


### Build step by step

In [3]:
from pqueens.models.model import Model
import numpy as np

np.random.seed(options["method"]["method_options"]["seed"])
num_samples = options["method"]["method_options"]["num_samples"]


# create surrogate models 
icm_model = Model.from_config_create_model("mf_gp_icm_surrogate_model", options)
nargp_model = Model.from_config_create_model("mf_gp_nargp_surrogate_model", options)

distribution_info = nargp_model.get_parameter_distribution_info()
numparams = len(distribution_info)
samples = np.zeros((num_samples, numparams))

i = 0
for distribution in distribution_info:
    # get appropriate random number generator
    random_number_generator = getattr(np.random, distribution['distribution'])
    # make a copy
    my_args = list(distribution['distribution_parameter'])
    my_args.extend([num_samples])
    samples [:, i] = random_number_generator(*my_args)
    i += 1
        
# update model
icm_model.update_model_from_sample_batch(samples)
nargp_model.update_model_from_sample_batch(samples)

/Applications/anaconda/envs/test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Compute Results

In [4]:
from pqueens.example_simulator_functions.perdikaris_1dsin_hifi import perdikaris_1dsin_hifi
from pqueens.example_simulator_functions.perdikaris_1dsin_lofi import perdikaris_1dsin_lofi

ref_outputs = perdikaris_1dsin_hifi(samples)
ref_outputs_lofi = perdikaris_1dsin_lofi(samples)

icm_outputs = icm_model.evaluate()
nargp_outputs = nargp_model.evaluate()
nargp_samples_lofi, nargp_samples_hifi = nargp_model.subordinate_iterator.samples
nargp_results_lofi, nargp_results_hifi = nargp_model.subordinate_iterator.outputs


/Users/jonas/work/adco/queens_code/pqueens/pqueens/iterators/scale_samples.py:44: UserWarning: Bounds of parameters are ignored when using                 scale_samples
  scale_samples")


Size of inputs in LHS(50, 1)
Inputs [[0.54074696]
 [0.15620227]
 [0.96050701]
 [0.69245781]
 [0.70170695]
 [0.03481537]
 [0.3467599 ]
 [0.2559659 ]
 [0.10587184]
 [0.21022685]
 [0.19826481]
 [0.47085289]
 [0.4115656 ]
 [0.99925297]
 [0.81452183]
 [0.07404968]
 [0.13618722]
 [0.60254121]
 [0.86645913]
 [0.89590372]
 [0.65539987]
 [0.38187964]
 [0.52061   ]
 [0.33780011]
 [0.29403934]
 [0.58720381]
 [0.27299928]
 [0.85032601]
 [0.72103363]
 [0.75062709]
 [0.36751166]
 [0.83951704]
 [0.51181667]
 [0.94156913]
 [0.05394031]
 [0.57645201]
 [0.42071885]
 [0.08718982]
 [0.7708127 ]
 [0.31591585]
 [0.90541665]
 [0.7927486 ]
 [0.48573083]
 [0.63044487]
 [0.66431642]
 [0.92877943]
 [0.01754746]
 [0.23003033]
 [0.17734145]
 [0.44931196]]
Size of outputs (50, 1)
Outputs [[ 0.85423776]
 [-0.706258  ]
 [-0.8374317 ]
 [-0.99224704]
 [-0.93692861]
 [ 0.76754718]
 [ 0.65164495]
 [ 0.14937832]
 [ 0.46243824]
 [-0.8412593 ]
 [-0.96362426]
 [-0.66876521]
 [-0.79499166]
 [-0.01877385]
 [ 0.99870924]
 [ 0.9

reconstraining parameters gp.mixed_noise.Gaussian_noise_0.variance
reconstraining parameters gp.mixed_noise.Gaussian_noise_1.variance


Optimization restart 1/30, f = -235.0445984249923
Optimization restart 2/30, f = 58.16845691510989
Optimization restart 3/30, f = 58.16845637783917
Optimization restart 4/30, f = 58.16847923948766
Optimization restart 5/30, f = 58.16845641167711
Optimization restart 6/30, f = 58.16845894789131
Optimization restart 7/30, f = 58.168458537678276
Optimization restart 8/30, f = 58.16845634027342
Optimization restart 9/30, f = 58.168457489596626
Optimization restart 10/30, f = 58.16847640121307
Optimization restart 11/30, f = 58.16845639805884
Optimization restart 12/30, f = 58.16846051459171
Optimization restart 13/30, f = 58.16845655437257
Optimization restart 14/30, f = 58.16845632647419
Optimization restart 15/30, f = 58.16845630163709
Optimization restart 16/30, f = 58.1684570576106
Optimization restart 17/30, f = 58.16846849550489
Optimization restart 18/30, f = 58.16845630103734
Optimization restart 19/30, f = 58.168456408569455
Optimization restart 20/30, f = 58.168457221713936
Optim

 /Users/jonas/work/adco/queens_code/pqueens/pqueens/iterators/scale_samples.py:44: UserWarning:Bounds of parameters are ignored when using                 scale_samples
reconstraining parameters GP_regression.Gaussian_noise.variance


Optimization restart 30/30, f = 58.168456732075796
Size of inputs in LHS(50, 1)
Inputs [[0.54074696]
 [0.15620227]
 [0.96050701]
 [0.69245781]
 [0.70170695]
 [0.03481537]
 [0.3467599 ]
 [0.2559659 ]
 [0.10587184]
 [0.21022685]
 [0.19826481]
 [0.47085289]
 [0.4115656 ]
 [0.99925297]
 [0.81452183]
 [0.07404968]
 [0.13618722]
 [0.60254121]
 [0.86645913]
 [0.89590372]
 [0.65539987]
 [0.38187964]
 [0.52061   ]
 [0.33780011]
 [0.29403934]
 [0.58720381]
 [0.27299928]
 [0.85032601]
 [0.72103363]
 [0.75062709]
 [0.36751166]
 [0.83951704]
 [0.51181667]
 [0.94156913]
 [0.05394031]
 [0.57645201]
 [0.42071885]
 [0.08718982]
 [0.7708127 ]
 [0.31591585]
 [0.90541665]
 [0.7927486 ]
 [0.48573083]
 [0.63044487]
 [0.66431642]
 [0.92877943]
 [0.01754746]
 [0.23003033]
 [0.17734145]
 [0.44931196]]
Size of outputs (50, 1)
Outputs [[ 0.85423776]
 [-0.706258  ]
 [-0.8374317 ]
 [-0.99224704]
 [-0.93692861]
 [ 0.76754718]
 [ 0.65164495]
 [ 0.14937832]
 [ 0.46243824]
 [-0.8412593 ]
 [-0.96362426]
 [-0.66876521]


reconstraining parameters GP_regression.Gaussian_noise.variance


Optimization restart 1/30, f = 7.430427097490558
Optimization restart 2/30, f = 7.430427017888361
Optimization restart 3/30, f = 7.4304269229674755
Optimization restart 4/30, f = 7.430426895932031
Optimization restart 5/30, f = 7.430426892192008


 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:167: RuntimeWarning:overflow encountered in true_divide
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/rbf.py:43: RuntimeWarning:overflow encountered in square
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:136: RuntimeWarning:overflow encountered in square
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:invalid value encountered in add


Optimization restart 6/30, f = 14.083260103535046
Optimization restart 7/30, f = 7.430426962780659
Optimization restart 8/30, f = 7.430426924925904
Optimization restart 9/30, f = 7.430426937898282
Optimization restart 10/30, f = 7.430433483337852
Optimization restart 11/30, f = 7.430426893264305
Optimization restart 12/30, f = 7.430426933007719
Optimization restart 13/30, f = 7.43042692026832
Optimization restart 14/30, f = 7.4304269154507505
Optimization restart 15/30, f = 7.43042693412609
Optimization restart 16/30, f = 7.430426901760253
Optimization restart 17/30, f = 7.430426910519479
Optimization restart 18/30, f = 7.430426930019933
Optimization restart 19/30, f = 7.430427153051095
Optimization restart 20/30, f = 7.430426888347189
Optimization restart 21/30, f = 7.430426929621226
Optimization restart 22/30, f = 7.430426897944194
Optimization restart 23/30, f = 7.430427060981184
Optimization restart 24/30, f = 7.43042707173265
Optimization restart 25/30, f = 7.430427074981293
Optim

In [5]:
def sort_ascending(array_1, array_2):
    """ Sort both arrays in increasing order based on array_1"""
    data = np.hstack((array_1,array_2))
    data = data[data[:,0].argsort()]
    return data[:,0], data[:,1]
    

### Plot results

In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected=True)

x_sorted, y_sorted = sort_ascending(samples,ref_outputs)
# Create a trace
reference_lofi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Hi-fi reference'
)

x_sorted, y_sorted = sort_ascending(nargp_samples_hifi,nargp_results_hifi)
trainingplot_hifi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers',
    name = 'Hi-fi training'
)

x_sorted, y_sorted = sort_ascending(samples,ref_outputs_lofi)
# Create a trace
reference = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Lo-fi reference'
)

x_sorted, y_sorted = sort_ascending(nargp_samples_lofi,nargp_results_lofi)
trainingplot_lofi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers',
    name = 'Lo-fi training'
)

x_sorted, y_sorted = sort_ascending(samples,icm_outputs)
icm = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'ICM'
)

x_sorted, y_sorted = sort_ascending(samples,nargp_outputs)
nargp = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'NARGP'
)



layout = dict(title = 'Multi-Fidelity Surrogate Modelling',
              xaxis = dict(title = 'Input'),
              yaxis = dict(title = 'Output'),
              )

fig = Figure(data=[reference,trainingplot_hifi, reference_lofi,trainingplot_lofi, icm, nargp], layout=layout)



iplot(fig)

### Single fidelity GP model

In [10]:
gp_model = Model.from_config_create_model("gp_surrogate_model", options)
# update model
gp_model.update_model_from_sample_batch(samples)
gp_outputs, gp_var = gp_model.evaluate()

gp_upper_raw = gp_outputs + 1.96*np.sqrt(gp_var)
gp_lower_raw = gp_outputs - 1.96*np.sqrt(gp_var)

input_x, gp_upper = sort_ascending(samples,gp_upper_raw)
input_x, gp_lower = sort_ascending(samples,gp_lower_raw)
gp_samples = gp_model.subordinate_iterator.samples
gp_results = gp_model.subordinate_iterator.outputs


/Users/jonas/work/adco/queens_code/pqueens/pqueens/iterators/scale_samples.py:44: UserWarning:

Bounds of parameters are ignored when using                 scale_samples

reconstraining parameters GP_regression.Gaussian_noise.variance


Size of inputs (23, 1)
Inputs [[0.22080245]
 [0.3462021 ]
 [0.05751985]
 [0.12152983]
 [0.38417305]
 [0.93964003]
 [0.52854074]
 [0.26197177]
 [0.97476522]
 [0.45256317]
 [0.4215641 ]
 [0.75130147]
 [0.1422101 ]
 [0.72435641]
 [0.89366757]
 [0.02244784]
 [0.63257507]
 [0.85816943]
 [0.57609752]
 [0.19299876]
 [0.82412458]
 [0.6832433 ]
 [0.48579541]]
Size of outputs (23, 1)
Outputs [[-5.35254106e-01]
 [-4.68352720e-01]
 [-1.33555017e+00]
 [-9.80778512e-03]
 [-5.37840410e-02]
 [-4.73202001e-01]
 [-3.82716199e-01]
 [-1.01203310e-01]
 [-1.54296463e-01]
 [-8.30284756e-01]
 [-8.41752057e-01]
 [-7.09008778e-04]
 [-2.23501255e-01]
 [-2.49006168e-01]
 [-1.06417205e-01]
 [-3.97940552e-01]
 [-2.79901878e-02]
 [-9.36974735e-02]
 [-7.43985628e-01]
 [-1.19803903e+00]
 [-5.41138198e-01]
 [-7.22635987e-01]
 [-1.13384019e-01]]
Optimization restart 1/30, f = 32.20349326758596
Optimization restart 2/30, f = 32.63558630142617
Optimization restart 3/30, f = 32.635586338174264
Optimization restart 4/30, f 

In [12]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected=True)


upper_bound = go.Scatter(
    name='Upper Bound',
    x=input_x,
    y=gp_upper,
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    showlegend=False,
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

x_sorted, y_sorted = sort_ascending(samples,gp_outputs)
gp = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    name = 'GP',
    mode='lines',
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

lower_bound = go.Scatter(
    name='Lower Bound',
    x=input_x,
    y=gp_lower,
    marker=dict(color="444"),
    line=dict(width=0),
    showlegend=False,
    mode='lines')

x_sorted, y_sorted = sort_ascending(gp_samples,gp_results)
trainingplot_hifi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers',
    name = 'Hi-fi training')

x_sorted, y_sorted = sort_ascending(samples,ref_outputs)
reference_hifi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Hi-fi reference')

layout = dict(title = 'GP Surrogate Modelling',
              xaxis = dict(title = 'Input'),
              yaxis = dict(title = 'Output'),
              )

fig = Figure(data=[lower_bound, gp, upper_bound, reference_hifi, trainingplot_hifi], layout=layout)

iplot(fig)